In [1]:
import os
import itertools
import sys
import torch

from typing import Any, Dict, List, Set

from detectron2.data import build_detection_train_loader
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.config import get_cfg
from detectron2.engine import DefaultTrainer, default_argument_parser, default_setup, launch
from detectron2.evaluation import COCOEvaluator
from detectron2.solver.build import maybe_add_gradient_clipping
from detectron2.data.datasets import register_coco_instances
import logging
from detectron2.utils.logger import setup_logger
from detectron2.utils import comm
from detectron2.engine.defaults import create_ddp_model
import weakref
from detectron2.engine.train_loop import AMPTrainer, SimpleTrainer
from ditod import add_vit_config
from ditod import DetrDatasetMapper
from ditod import MyDetectionCheckpointer, ICDAREvaluator
from ditod import MyTrainer


/home/saeed/Downloads/yes/envs/rcnnlmv3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def setup(args):
    """
    Create configs and perform basic setups.
    """
    cfg = get_cfg()
    # add_coat_config(cfg)
    add_vit_config(cfg)
    cfg.merge_from_file(args.config_file)
    cfg.merge_from_list(args.opts)
    cfg.MODEL.DEVICE = 'cpu'
    cfg.freeze()
    default_setup(cfg, args)
    return cfg

In [3]:
import argparse
import sys
import os
from detectron2.modeling import build_model
def default_argument_parser(epilog=None):

    parser = argparse.ArgumentParser(
        epilog=epilog,
#         or f"""
# Examples:

# Run on single machine:
#     $ {sys.argv[0]} --num-gpus 8 --config-file cfg.yaml

# Change some config options:
#     $ {sys.argv[0]} --config-file cfg.yaml MODEL.WEIGHTS /path/to/weight.pth SOLVER.BASE_LR 0.001

# Run on multiple machines:
#     (machine0)$ {sys.argv[0]} --machine-rank 0 --num-machines 2 --dist-url <URL> [--other-flags]
#     (machine1)$ {sys.argv[0]} --machine-rank 1 --num-machines 2 --dist-url <URL> [--other-flags]
# """,
        formatter_class=argparse.RawDescriptionHelpFormatter,
    )
    parser.add_argument("--config-file", default="/home/saeed/Walnut/lmv3rcnn_image-only/layoutlmv3/examples/object_detection/cascade_layoutlmv3.yaml", metavar="FILE", help="path to config file")
    parser.add_argument(
        "--resume",
        action="store_true",
        help="Whether to attempt to resume from the checkpoint directory. "
        "See documentation of `DefaultTrainer.resume_or_load()` for what it means.",
    )
    parser.add_argument("--eval-only", action="store_true", help="perform evaluation only")
    parser.add_argument("--num-gpus", type=int, default=2, help="number of gpus *per machine*")
    parser.add_argument("--num-machines", type=int, default=1, help="total number of machines")
    parser.add_argument(
        "--machine-rank", type=int, default=0, help="the rank of this machine (unique per machine)"
    )

    # PyTorch still may leave orphan processes in multi-gpu training.
    # Therefore we use a deterministic way to obtain port,
    # so that users are aware of orphan processes by seeing the port occupied.
    port = 2**15 + 2**14 + hash(os.getuid() if sys.platform != "win32" else 1) % 2**14
    parser.add_argument(
        "--dist-url",
        default="tcp://127.0.0.1:{}".format(port),
        help="initialization URL for pytorch distributed backend. See "
        "https://pytorch.org/docs/stable/distributed.html for details.",
    )
    parser.add_argument(
        "opts",
        help="""
Modify config options at the end of the command. For Yacs configs, use
space-separated "PATH.KEY VALUE" pairs.
For python-based LazyConfig, use "path.key=value".
        """.strip(),
        default=None,
        nargs=argparse.REMAINDER,
    )
    return parser

In [4]:
parser1 = default_argument_parser()
parser1.add_argument("--debug", action="store_true", help="enable debug mode")
args = parser1.parse_args("")
configs = setup(args)

[08/29 17:06:44 detectron2]: Rank of current process: 0. World size: 1


[08/29 17:06:44 detectron2]: Environment info:
-------------------------------  ----------------------------------------------------------------------------------------
sys.platform                     linux
Python                           3.11.4 (main, Jul  5 2023, 13:45:01) [GCC 11.2.0]
numpy                            1.25.2
detectron2                       0.6 @/home/saeed/Downloads/yes/envs/rcnnlmv3/lib/python3.11/site-packages/detectron2
Compiler                         GCC 12.3
CUDA compiler                    not available
DETECTRON2_ENV_MODULE            <not set>
PyTorch                          2.0.1 @/home/saeed/Downloads/yes/envs/rcnnlmv3/lib/python3.11/site-packages/torch
PyTorch debug build              False
torch._C._GLIBCXX_USE_CXX11_ABI  False
GPU available                    Yes
GPU 0                            NVIDIA GeForce RTX 3050 Ti Laptop GPU (arch=8.6)
Driver version                   535.104.05
CUDA_HOME                        /usr/local/cuda
Pillow        

In [5]:
from detectron2.data.datasets import register_coco_instances
register_coco_instances(
      "publaynet_train",
      {},
      "/home/saeed/Walnut/lmv3rcnn_image-only/layoutlmv3/Fintab.v2i.coco/train/_annotations.coco.json",
      "/home/saeed/Walnut/lmv3rcnn_image-only/layoutlmv3/Fintab.v2i.coco/train"
  )

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: saeed11b95 (walnut). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/saeed/.netrc


True

In [8]:
trainer = MyTrainer(configs)

WARNING [08/29 17:06:56 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[08/29 17:06:56 d2.data.datasets.coco]: Loaded 130 images in COCO format from /home/saeed/Walnut/lmv3rcnn_image-only/layoutlmv3/Fintab.v2i.coco/train/_annotations.coco.json
[08/29 17:06:56 d2.data.build]: Distribution of instances among all 7 categories:
|   category    | #instances   |   category    | #instances   |   category   | #instances   |
|:-------------:|:-------------|:-------------:|:-------------|:------------:|:-------------|
|    Tables     | 0            |     table     | 130          | table column | 507          |
| table colum.. | 107          | table proje.. | 162          |  table row   | 1294         |
| table spann.. | 70           |               |              |              |              |
|     total     | 2270         |               |              |              |              |
[08/29 17:06:56 d2.data.build]: Using train

In [9]:
trainer.train()

/home/saeed/Downloads/yes/envs/rcnnlmv3/lib/python3.11/site-packages/transformers/modeling_utils.py:900: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
